<a href="https://colab.research.google.com/github/QuinnWass/CPSC-4310-01/blob/master/DAP5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DAP5

In [0]:
import pandas as pd
import re
import nltk
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import numpy as np
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from tqdm import tqdm, tqdm_gui, tqdm_notebook
import tensorflow as tf
from tensorflow import keras

%matplotlib inline 
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize = 14)
mpl.rc('xtick', labelsize = 12)
mpl.rc('ytick', labelsize = 12)


from scipy import stats

In [0]:
parts_of_speech = ['CC', 'CD', 'DT', 'EX', 'FW', 'IN', 'JJ',
                    'JJR', 'JJS', 'LS', 'MD', 'NN', 'NNS', 'NNP',
                    'NNPS', 'PDT', 'PRP$', 'RB', 'RBR', 'PRP', 'RBS',
                    'RP', 'TO', 'UH', 'VB', 'VBD', 'VBG', 'VBN', 'VBP',
                    'VBZ', 'WDT', 'WP', 'WP$', 'WRB' ]



#Load dataset into df
dataset = 'https://raw.githubusercontent.com/QuinnWass/CPSC-4310-01/master/FakeNewsClassifier/processeddata/cb_full_parsed.csv'
df = pd.read_csv(dataset)


#this is to get rid  https://seattleu.zoom.us/j/5664223549 of the XX.1 thing we have going on for some reason when
#the dataframe is loaded.
df = df.drop(columns = parts_of_speech)
parts_of_speech_dict = {'CC.1':'CC', 'CD.1':'CD', 'DT.1':'DT', 'EX.1':'EX', 'FW.1':'FW', 'IN.1':'IN', 'JJ.1':'JJ', 'JJR.1':'JJR', 'JJS.1':'JJS', 'LS.1':'LS', 'MD.1':'MD',
                   'NN.1':'NN', 'NNS.1':'NNS', 'NNP.1':'NNP', \
                    'NNPS.1':'NNPS', 'PDT.1':'PDT', 'PRP$.1':'PRP$', 'RB.1':'RB', 'RBR.1':'RBR', 'PRP.1':'PRP', 'RBS.1':'RBS', \
                    'RP.1':'RP', 'TO.1':'TO', 'UH.1':'UH', 'VB.1':'VB', 'VBD.1':'VBD', 'VBG.1':'VBG', 'VBN.1':'VBN', 'VBP.1':'VBP', \
                    'VBZ.1':'VBZ', 'WDT.1':'WDT', 'WP.1':'WP', 'WP$.1':'WP$', 'WRB.1':'WRB' }
df = df.rename(columns=parts_of_speech_dict)



df.classification = pd.Categorical(df.classification)
df['code'] = df.classification.cat.codes

#appending additional features to the parts of speech array
#as our exploratory analysis has shown they may be a factor
features = parts_of_speech + ['word_count', 'title_sentiment', 'sentiment']
X_df = df[features]
Y_df = df['code']

X = X_df.values
y = Y_df.values

In [0]:
df.describe()

,Unnamed: 0,sentiment,word_count,title_word_count,title_sentiment,CC,CD,DT,EX,FW,IN,JJ,JJR,JJS,LS,MD,NN,NNS,NNP,NNPS,PDT,PRP$,RB,RBR,PRP,RBS,RP,TO,UH,VB,VBD,VBG,VBN,VBP,VBZ,WDT,WP,WP$,WRB,code
count,44267.00000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.0,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000,44267.000000
mean,22133.00000,0.016331,252.014977,11.576841,-0.124848,0.889037,2.749723,4.478325,0.096731,0.327806,6.850792,47.862810,0.676554,0.765807,0.0,2.605349,110.977455,7.749994,0.499605,0.074977,0.061716,0.278560,11.879233,0.467549,2.704453,0.044661,0.223643,0.109924,0.018818,5.376578,16.949714,10.382791,6.173944,7.937606,2.191113,0.160503,0.192581,0.081212,0.269998,0.483792
std,12778.92652,0.841605,216.469833,4.551793,0.427399,1.569163,3.401539,4.763000,0.341811,0.930322,7.026680,43.488887,1.159265,1.223750,0.0,3.265678,97.027810,7.335827,1.095131,0.337298,0.252462,0.681485,12.455623,0.822224,3.610236,0.222056,0.548096,0.439312,0.155570,5.425026,14.853689,9.974610,6.380545,7.524627,2.631081,0.458720,0.572871,0.313085,0.667432,0.499743
min,0.00000,-1.000000,1.000000,1.000000,-0.975000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,11066.50000,-0.936000,132.000000,8.000000,-0.476700,0.000000,0.000000,2.000000,0.000000,0.000000,3.000000,23.000000,0.000000,0.000000,0.0,0.000000,58.000000,3.000000,0.000000,0.000000,0.000000,0.000000,4.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,2.000000,7.000000,4.000000,2.000000,3.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,22133.00000,0.077200,222.000000,10.000000,0.000000,0.000000,2.000000,4.000000,0.000000,0.000000,6.000000,40.000000,0.000000,0.000000,0.0,2.000000,96.000000,6.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,4.000000,14.000000,9.000000,5.000000,6.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,33199.50000,0.926000,314.000000,14.000000,0.102700,1.000000,4.000000,6.000000,0.000000,0.000000,9.000000,60.000000,1.000000,1.000000,0.0,4.000000,139.000000,10.000000,1.000000,0.000000,0.000000,0.000000,16.000000,1.000000,4.000000,0.000000,0.000000,0.000000,0.000000,7.000000,22.000000,14.000000,8.000000,11.000000,3.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,44266.00000,1.000000,5193.000000,43.000000,0.970500,68.000000,77.000000,165.000000,11.000000,37.000000,175.000000,1145.000000,37.000000,53.000000,0.0,95.000000,3028.000000,211.000000,40.000000,9.000000,5.000000,26.000000,280.000000,11.000000,118.000000,8.000000,16.000000,16.000000,5.000000,114.000000,351.000000,270.000000,185.000000,145.000000,97.000000,14.000000,13.000000,7.000000,14.000000,1.000000


In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y,
                                                    test_size = 0.33,
                                                    random_state=42)


In [0]:
# Import `StandardScaler` from `sklearn.preprocessing`
from sklearn.preprocessing import StandardScaler

# Define the scaler 
scaler = StandardScaler().fit(X_train)

# Scale the train set
X_train = scaler.transform(X_train)

# Scale the test set
X_test = scaler.transform(X_test)

## Model A

In [0]:
# construct a model

modelA = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
modelA.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelA.fit(X_train, y_train, epochs=10, batch_size=50, verbose=1)

print('Evaluate')
modelA.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
594/594 [==============================] - 1s 2ms/step - loss: 0.4399 - accuracy: 0.7976
Epoch 2/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3421 - accuracy: 0.8551
Epoch 3/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3289 - accuracy: 0.8585
Epoch 4/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3216 - accuracy: 0.8643
Epoch 5/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3153 - accuracy: 0.8646
Epoch 6/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3117 - accuracy: 0.8657
Epoch 7/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3065 - accuracy: 0.8684
Epoch 8/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3041 - accuracy: 0.8714
Epoch 9/10
594/594 [==============================] - 1s 2ms/step - loss: 0.3010 - accuracy: 0.8738
Epoch 10/10
594/594 [==============================] - 1s 2ms/step - loss: 0.2975 - accuracy: 0.8735

[0.32164573669433594, 0.8645355701446533]

## Model B Extra layer, more nodes

In [0]:
# construct a model

modelB = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
modelB.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelB.fit(X_train, y_train, epochs=10, batch_size=100, verbose=1)

print('Evaluate')
modelB.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
297/297 [==============================] - 1s 2ms/step - loss: 0.4332 - accuracy: 0.8049
Epoch 2/10
297/297 [==============================] - 1s 2ms/step - loss: 0.3462 - accuracy: 0.8535
Epoch 3/10
297/297 [==============================] - 1s 2ms/step - loss: 0.3303 - accuracy: 0.8605
Epoch 4/10
297/297 [==============================] - 1s 2ms/step - loss: 0.3211 - accuracy: 0.8660
Epoch 5/10
297/297 [==============================] - 1s 2ms/step - loss: 0.3110 - accuracy: 0.8708
Epoch 6/10
297/297 [==============================] - 1s 3ms/step - loss: 0.3063 - accuracy: 0.8725
Epoch 7/10
297/297 [==============================] - 1s 3ms/step - loss: 0.3002 - accuracy: 0.8750
Epoch 8/10
297/297 [==============================] - 1s 2ms/step - loss: 0.2956 - accuracy: 0.8765
Epoch 9/10
297/297 [==============================] - 1s 2ms/step - loss: 0.2894 - accuracy: 0.8801
Epoch 10/10
297/297 [==============================] - 1s 2ms/step - loss: 0.2836 - accuracy: 0.8824

[0.31907206773757935, 0.8633034229278564]

## Model C Wide



In [0]:
# construct a model
modelB = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelB.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelB.fit(X_train, y_train, epochs=100, batch_size=100, verbose=1)

print('Evaluate')
modelB.evaluate(X_test, y_test, verbose=1)

Epoch 1/100
297/297 [==============================] - 2s 7ms/step - loss: 0.4304 - accuracy: 0.8037
Epoch 2/100
297/297 [==============================] - 2s 7ms/step - loss: 0.3482 - accuracy: 0.8540
Epoch 3/100
297/297 [==============================] - 2s 7ms/step - loss: 0.3307 - accuracy: 0.8612
Epoch 4/100
297/297 [==============================] - 2s 7ms/step - loss: 0.3114 - accuracy: 0.8697
Epoch 5/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2984 - accuracy: 0.8747
Epoch 6/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2868 - accuracy: 0.8795
Epoch 7/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2746 - accuracy: 0.8854
Epoch 8/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2610 - accuracy: 0.8908
Epoch 9/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2439 - accuracy: 0.8972
Epoch 10/100
297/297 [==============================] - 2s 7ms/step - loss: 0.2270 - accura

[0.9663607478141785, 0.8868505954742432]

As can be seen, this model grossly overfits and has a high loss during evaluation

## Model C Reduced Epochs

In [0]:
# construct a model
modelB = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelB.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelB.fit(X_train, y_train, epochs=25, batch_size=100, verbose=1)

print('Evaluate')
modelB.evaluate(X_test, y_test, verbose=1)

Epoch 1/25
297/297 [==============================] - 2s 8ms/step - loss: 0.4323 - accuracy: 0.8062
Epoch 2/25
297/297 [==============================] - 2s 8ms/step - loss: 0.3467 - accuracy: 0.8547
Epoch 3/25
297/297 [==============================] - 2s 8ms/step - loss: 0.3272 - accuracy: 0.8626
Epoch 4/25
297/297 [==============================] - 2s 8ms/step - loss: 0.3145 - accuracy: 0.8678
Epoch 5/25
297/297 [==============================] - 2s 8ms/step - loss: 0.3019 - accuracy: 0.8734
Epoch 6/25
297/297 [==============================] - 2s 8ms/step - loss: 0.2943 - accuracy: 0.8757
Epoch 7/25
297/297 [==============================] - 2s 8ms/step - loss: 0.2803 - accuracy: 0.8812
Epoch 8/25
297/297 [==============================] - 2s 8ms/step - loss: 0.2630 - accuracy: 0.8882
Epoch 9/25
297/297 [==============================] - 2s 8ms/step - loss: 0.2487 - accuracy: 0.8942
Epoch 10/25
297/297 [==============================] - 2s 8ms/step - loss: 0.2327 - accuracy: 0.9010

[0.7797792553901672, 0.8794578909873962]

As can be seen, the loss is still high

## Model D
Reducing batch size of model B

In [0]:
# construct a model

modelD = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
modelD.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelD.fit(X_train, y_train, epochs=10, batch_size=1, verbose=1)

print('Evaluate')
modelD.evaluate(X_test, y_test, verbose=1)

Epoch 1/10
670/670 [==============================] - 2s 3ms/step - loss: 0.4241 - accuracy: 0.8254
Epoch 2/10
670/670 [==============================] - 2s 3ms/step - loss: 0.2102 - accuracy: 0.9269
Epoch 3/10
670/670 [==============================] - 2s 3ms/step - loss: 0.1588 - accuracy: 0.9552
Epoch 4/10
670/670 [==============================] - 2s 3ms/step - loss: 0.1222 - accuracy: 0.9537
Epoch 5/10
670/670 [==============================] - 2s 3ms/step - loss: 0.0772 - accuracy: 0.9746
Epoch 6/10
670/670 [==============================] - 2s 3ms/step - loss: 0.1137 - accuracy: 0.9627
Epoch 7/10
670/670 [==============================] - 2s 3ms/step - loss: 0.0794 - accuracy: 0.9716
Epoch 8/10
670/670 [==============================] - 2s 3ms/step - loss: 0.0514 - accuracy: 0.9776
Epoch 9/10
670/670 [==============================] - 2s 3ms/step - loss: 0.0487 - accuracy: 0.9896
Epoch 10/10
670/670 [==============================] - 2s 3ms/step - loss: 0.0490 - accuracy: 0.9851

[0.3083204925060272, 0.9363636374473572]

# Model E

In [0]:
# construct a model

modelE = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='sigmoid', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(2 * len(X_df.columns), activation='sigmoid'),
    keras.layers.Dense(3 * len(X_df.columns), activation='swish'),
    keras.layers.Dense(2 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(3 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
modelE.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 
                                                                      tf.keras.metrics.AUC()])

modelE.fit(X_train, y_train, epochs=10, batch_size=15, validation_split=0.2, verbose=1)

print('Evaluate')
modelE.evaluate(X_test, y_test, verbose=1)


Epoch 1/10
1370/1370 [==============================] - 6s 5ms/step - loss: 0.4310 - accuracy: 0.7983 - auc_22: 0.8807 - val_loss: 0.3576 - val_accuracy: 0.8497 - val_auc_22: 0.9258
Epoch 2/10
1370/1370 [==============================] - 6s 4ms/step - loss: 0.3693 - accuracy: 0.8430 - auc_22: 0.9143 - val_loss: 0.4215 - val_accuracy: 0.8218 - val_auc_22: 0.9294
Epoch 3/10
1370/1370 [==============================] - 7s 5ms/step - loss: 0.3580 - accuracy: 0.8469 - auc_22: 0.9194 - val_loss: 0.3830 - val_accuracy: 0.8403 - val_auc_22: 0.9297
Epoch 4/10
1370/1370 [==============================] - 6s 5ms/step - loss: 0.3510 - accuracy: 0.8501 - auc_22: 0.9227 - val_loss: 0.3347 - val_accuracy: 0.8545 - val_auc_22: 0.9320
Epoch 5/10
1370/1370 [==============================] - 6s 4ms/step - loss: 0.3509 - accuracy: 0.8497 - auc_22: 0.9226 - val_loss: 0.3574 - val_accuracy: 0.8432 - val_auc_22: 0.9336
Epoch 6/10
1370/1370 [==============================] - 6s 4ms/step - loss: 0.3457 - accur

[0.3388303220272064, 0.8586027026176453, 0.9340227842330933]

# Model F

In [0]:
# construct a model

modelE = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='sigmoid', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='sigmoid'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(X_df.columns), activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(1, activation ='sigmoid')
])



modelE.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 
                                                                      tf.keras.metrics.AUC()])

modelE.fit(X_train, y_train, epochs=20, batch_size=40, validation_steps = 4, validation_split=0.2, verbose=1)

print('Evaluate')
modelE.evaluate(X_test, y_test, verbose=1)


Epoch 1/20
594/594 [==============================] - 3s 5ms/step - loss: 0.4843 - accuracy: 0.7584 - auc_23: 0.8459 - val_loss: 0.2736 - val_accuracy: 0.9125 - val_auc_23: 0.9521
Epoch 2/20
594/594 [==============================] - 3s 5ms/step - loss: 0.3753 - accuracy: 0.8409 - auc_23: 0.9116 - val_loss: 0.2911 - val_accuracy: 0.8875 - val_auc_23: 0.9529
Epoch 3/20
594/594 [==============================] - 3s 5ms/step - loss: 0.3603 - accuracy: 0.8463 - auc_23: 0.9185 - val_loss: 0.2698 - val_accuracy: 0.9062 - val_auc_23: 0.9530
Epoch 4/20
594/594 [==============================] - 3s 5ms/step - loss: 0.3526 - accuracy: 0.8493 - auc_23: 0.9222 - val_loss: 0.2858 - val_accuracy: 0.8875 - val_auc_23: 0.9571
Epoch 5/20
594/594 [==============================] - 3s 5ms/step - loss: 0.3456 - accuracy: 0.8516 - auc_23: 0.9254 - val_loss: 0.2622 - val_accuracy: 0.9062 - val_auc_23: 0.9604
Epoch 6/20
594/594 [==============================] - 3s 5ms/step - loss: 0.3397 - accuracy: 0.8537 

[0.3163735270500183, 0.8630980849266052, 0.9381943345069885]

# Model G

In [0]:
# construct a model
#deleted 2 layers from initial relu bunch

modelG = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='sigmoid', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='relu'),
    keras.layers.Dense(len(X_df.columns), activation='sigmoid'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(len(X_df.columns), activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(1, activation ='sigmoid')
])



modelG.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', 
                                                                      tf.keras.metrics.AUC()])

modelG.fit(X_train, y_train, epochs=20, batch_size=40, validation_steps = 4, validation_split=0.2, verbose=1)

print('Evaluate')
modelG.evaluate(X_test, y_test, verbose=1)

Epoch 1/20
594/594 [==============================] - 3s 5ms/step - loss: 0.4787 - accuracy: 0.7542 - auc_24: 0.8475 - val_loss: 0.2912 - val_accuracy: 0.8938 - val_auc_24: 0.9528
Epoch 2/20
594/594 [==============================] - 3s 4ms/step - loss: 0.3757 - accuracy: 0.8398 - auc_24: 0.9111 - val_loss: 0.2788 - val_accuracy: 0.8875 - val_auc_24: 0.9578
Epoch 3/20
594/594 [==============================] - 3s 4ms/step - loss: 0.3630 - accuracy: 0.8464 - auc_24: 0.9172 - val_loss: 0.2773 - val_accuracy: 0.9000 - val_auc_24: 0.9560
Epoch 4/20
594/594 [==============================] - 3s 4ms/step - loss: 0.3561 - accuracy: 0.8470 - auc_24: 0.9205 - val_loss: 0.2636 - val_accuracy: 0.8938 - val_auc_24: 0.9575
Epoch 5/20
594/594 [==============================] - 3s 4ms/step - loss: 0.3508 - accuracy: 0.8499 - auc_24: 0.9229 - val_loss: 0.2742 - val_accuracy: 0.9000 - val_auc_24: 0.9572
Epoch 6/20
594/594 [==============================] - 3s 4ms/step - loss: 0.3453 - accuracy: 0.8521 

[0.3409321904182434, 0.8471490144729614, 0.9360639452934265]

# model G


In [0]:
# construct a model
#deleted 2 layers from initial relu bunch

modelG = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='sigmoid', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(len(X_df.columns), activation='swish'),
    keras.layers.Dense(len(X_df.columns), activation='swish'),
    keras.layers.Dense(2 * len(X_df.columns), activation='sigmoid'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(2*len(X_df.columns), activation='swish'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(40, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(20, activation='swish'),
    keras.layers.Dense(1, activation ='sigmoid')
])



modelG.compile(loss='kld', optimizer='adam', metrics=['accuracy', 
                                                                      tf.keras.metrics.AUC()])

modelG.fit(X_train, y_train, epochs=30, batch_size=40, validation_steps = 4, validation_split=0.2, verbose=1)

print('Evaluate')
modelG.evaluate(X_test, y_test, verbose=1)

Epoch 1/30
594/594 [==============================] - 3s 5ms/step - loss: 0.0122 - accuracy: 0.4854 - auc_15: 0.4967 - val_loss: -8.8650e-07 - val_accuracy: 0.4500 - val_auc_15: 0.5000
Epoch 2/30
594/594 [==============================] - 3s 5ms/step - loss: -7.7919e-07 - accuracy: 0.4857 - auc_15: 0.5000 - val_loss: -8.8650e-07 - val_accuracy: 0.4500 - val_auc_15: 0.5000
Epoch 3/30
594/594 [==============================] - 3s 4ms/step - loss: -7.8766e-07 - accuracy: 0.4857 - auc_15: 0.5000 - val_loss: -8.8650e-07 - val_accuracy: 0.4500 - val_auc_15: 0.5000
Epoch 4/30
594/594 [==============================] - 3s 5ms/step - loss: -7.9133e-07 - accuracy: 0.4857 - auc_15: 0.5000 - val_loss: -8.8650e-07 - val_accuracy: 0.4500 - val_auc_15: 0.5000
Epoch 5/30
594/594 [==============================] - 3s 5ms/step - loss: -7.9653e-07 - accuracy: 0.4857 - auc_15: 0.5000 - val_loss: -8.8650e-07 - val_accuracy: 0.4500 - val_auc_15: 0.5000
Epoch 6/30
594/594 [==============================] - 3

KeyboardInterrupt: ignored

In [0]:
# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelK.fit(X_train, y_train, epochs=25, batch_size=100, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)

Epoch 1/20
297/297 [==============================] - 4s 15ms/step - loss: 0.4330 - accuracy: 0.8039
Epoch 2/20
297/297 [==============================] - 4s 14ms/step - loss: 0.3488 - accuracy: 0.8521
Epoch 3/20
297/297 [==============================] - 4s 14ms/step - loss: 0.3293 - accuracy: 0.8611
Epoch 4/20
297/297 [==============================] - 4s 14ms/step - loss: 0.3138 - accuracy: 0.8675
Epoch 5/20
297/297 [==============================] - 4s 14ms/step - loss: 0.3029 - accuracy: 0.8717
Epoch 6/20
297/297 [==============================] - 4s 15ms/step - loss: 0.2877 - accuracy: 0.8792
Epoch 7/20
297/297 [==============================] - 4s 14ms/step - loss: 0.2765 - accuracy: 0.8832
Epoch 8/20
297/297 [==============================] - 4s 14ms/step - loss: 0.2626 - accuracy: 0.8872
Epoch 9/20
297/297 [==============================] - 4s 14ms/step - loss: 0.2497 - accuracy: 0.8942
Epoch 10/20
297/297 [==============================] - 4s 14ms/step - loss: 0.2318 - accura

[0.5963054895401001, 0.8661783933639526]

# Model K

In [0]:
# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelK.fit(X_train, y_train, epochs=25, batch_size=242, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)


#It can be seen in the epoch's below that as the loss function yeilds
#lower and lower results that the model is obviously being overfit. 

#the significant drop in accuracy on the testing set is also indicative of 
#overfitting

Epoch 1/25
123/123 [==============================] - 5s 37ms/step - loss: 0.4872 - accuracy: 0.7752
Epoch 2/25
123/123 [==============================] - 4s 36ms/step - loss: 0.3623 - accuracy: 0.8454
Epoch 3/25
123/123 [==============================] - 5s 37ms/step - loss: 0.3367 - accuracy: 0.8583
Epoch 4/25
123/123 [==============================] - 5s 37ms/step - loss: 0.3144 - accuracy: 0.8672
Epoch 5/25
123/123 [==============================] - 5s 38ms/step - loss: 0.3035 - accuracy: 0.8725
Epoch 6/25
123/123 [==============================] - 5s 37ms/step - loss: 0.2877 - accuracy: 0.8795
Epoch 7/25
123/123 [==============================] - 5s 37ms/step - loss: 0.2746 - accuracy: 0.8843
Epoch 8/25
123/123 [==============================] - 4s 36ms/step - loss: 0.2592 - accuracy: 0.8912
Epoch 9/25
123/123 [==============================] - 4s 36ms/step - loss: 0.2426 - accuracy: 0.8972
Epoch 10/25
123/123 [==============================] - 4s 36ms/step - loss: 0.2290 - accura

[0.7718998193740845, 0.8754192590713501]

To correct for overfitting, adding a dropout layer

In [0]:
# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dropout(0.5,
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelK.fit(X_train, y_train, epochs=25, batch_size=242, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)


Epoch 1/25
123/123 [==============================] - 5s 37ms/step - loss: 0.4942 - accuracy: 0.7669
Epoch 2/25
123/123 [==============================] - 5s 37ms/step - loss: 0.3627 - accuracy: 0.8454
Epoch 3/25
123/123 [==============================] - 5s 37ms/step - loss: 0.3359 - accuracy: 0.8592
Epoch 4/25
123/123 [==============================] - 5s 37ms/step - loss: 0.3140 - accuracy: 0.8677
Epoch 5/25
123/123 [==============================] - 5s 38ms/step - loss: 0.3057 - accuracy: 0.8726
Epoch 6/25
123/123 [==============================] - 5s 38ms/step - loss: 0.2891 - accuracy: 0.8801
Epoch 7/25
123/123 [==============================] - 5s 37ms/step - loss: 0.2755 - accuracy: 0.8845
Epoch 8/25
123/123 [==============================] - 5s 37ms/step - loss: 0.2607 - accuracy: 0.8916
Epoch 9/25
123/123 [==============================] - 4s 36ms/step - loss: 0.2454 - accuracy: 0.8975
Epoch 10/25
123/123 [==============================] - 4s 36ms/step - loss: 0.2240 - accura

[0.7321678996086121, 0.8752139210700989]

In [0]:
#Reducing number of Epochs

# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

modelK.fit(X_train, y_train, epochs=18, batch_size=242, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)


Epoch 1/18
123/123 [==============================] - 5s 37ms/step - loss: 0.4962 - accuracy: 0.7671
Epoch 2/18
123/123 [==============================] - 4s 36ms/step - loss: 0.3679 - accuracy: 0.8452
Epoch 3/18
123/123 [==============================] - 4s 37ms/step - loss: 0.3406 - accuracy: 0.8566
Epoch 4/18
123/123 [==============================] - 5s 37ms/step - loss: 0.3245 - accuracy: 0.8649
Epoch 5/18
123/123 [==============================] - 5s 37ms/step - loss: 0.3070 - accuracy: 0.8737
Epoch 6/18
123/123 [==============================] - 5s 37ms/step - loss: 0.2960 - accuracy: 0.8771
Epoch 7/18
123/123 [==============================] - 5s 37ms/step - loss: 0.2824 - accuracy: 0.8824
Epoch 8/18
123/123 [==============================] - 5s 37ms/step - loss: 0.2658 - accuracy: 0.8893
Epoch 9/18
123/123 [==============================] - 5s 37ms/step - loss: 0.2497 - accuracy: 0.8971
Epoch 10/18
123/123 [==============================] - 4s 36ms/step - loss: 0.2408 - accura

[0.5329168438911438, 0.8644670844078064]

In [11]:
#Reducing number of Epochs

# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(len(X_df.columns), activation='relu', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(100 * len(X_df.columns), activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy', ])

modelK.fit(X_train, y_train, epochs=20, batch_size=100, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)


Epoch 1/20
297/297 [==============================] - 5s 18ms/step - loss: 0.4573 - accuracy: 0.7917
Epoch 2/20
297/297 [==============================] - 5s 18ms/step - loss: 0.3583 - accuracy: 0.8513
Epoch 3/20
297/297 [==============================] - 5s 18ms/step - loss: 0.3337 - accuracy: 0.8611
Epoch 4/20
297/297 [==============================] - 5s 18ms/step - loss: 0.3141 - accuracy: 0.8666
Epoch 5/20
297/297 [==============================] - 5s 18ms/step - loss: 0.3048 - accuracy: 0.8715
Epoch 6/20
297/297 [==============================] - 5s 18ms/step - loss: 0.2919 - accuracy: 0.8782
Epoch 7/20
297/297 [==============================] - 5s 18ms/step - loss: 0.2786 - accuracy: 0.8826
Epoch 8/20
297/297 [==============================] - 5s 18ms/step - loss: 0.2625 - accuracy: 0.8873
Epoch 9/20
297/297 [==============================] - 5s 18ms/step - loss: 0.2487 - accuracy: 0.8945
Epoch 10/20
297/297 [==============================] - 5s 18ms/step - loss: 0.2319 - accura

[0.55209881067276, 0.871722936630249]

In [74]:
#Reducing number of Epochs

# construct a model
modelK = keras.models.Sequential([
    keras.layers.Dense(2*len(X_df.columns), activation='swish', input_shape=(len(X_df.columns),)),
    keras.layers.Dense(100, activation = 'swish'),
    keras.layers.Dropout(0.25),
    keras.layers.Dense(100,activation='swish'),
    keras.layers.Dropout(0.5),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(40, activation = 'swish'),
    keras.layers.Dense(len(X_df.columns), activation = 'sigmoid'),
    keras.layers.Dense(1, activation='sigmoid')
])

modelK.compile(loss='BinaryCrossentropy', optimizer='nadam', metrics=['accuracy', tf.keras.metrics.AUC()])

modelK.fit(X_train, y_train, epochs=50, batch_size=242, validation_steps=4, validation_split=0.12, 
           validation_batch_size=100, verbose=1)

print('Evaluate')
modelK.evaluate(X_test, y_test, verbose=1)


Epoch 1/50
108/108 [==============================] - 1s 12ms/step - loss: 0.4654 - accuracy: 0.7680 - auc_22: 0.8565 - val_loss: 0.3785 - val_accuracy: 0.8325 - val_auc_22: 0.9091
Epoch 2/50
108/108 [==============================] - 1s 9ms/step - loss: 0.3633 - accuracy: 0.8470 - auc_22: 0.9167 - val_loss: 0.3479 - val_accuracy: 0.8400 - val_auc_22: 0.9246
Epoch 3/50
108/108 [==============================] - 1s 10ms/step - loss: 0.3490 - accuracy: 0.8510 - auc_22: 0.9238 - val_loss: 0.3451 - val_accuracy: 0.8450 - val_auc_22: 0.9271
Epoch 4/50
108/108 [==============================] - 1s 10ms/step - loss: 0.3437 - accuracy: 0.8554 - auc_22: 0.9260 - val_loss: 0.3290 - val_accuracy: 0.8575 - val_auc_22: 0.9319
Epoch 5/50
108/108 [==============================] - 1s 11ms/step - loss: 0.3378 - accuracy: 0.8585 - auc_22: 0.9286 - val_loss: 0.3289 - val_accuracy: 0.8550 - val_auc_22: 0.9330
Epoch 6/50
108/108 [==============================] - 1s 9ms/step - loss: 0.3338 - accuracy: 0.8

[0.31299808621406555, 0.871722936630249, 0.9404857754707336]